In [1]:
import numpy as np
import pandas as pd
from datetime import datetime
from dateutil.relativedelta import relativedelta
from scipy import optimize

# Change the representation of float format to show the values as they are (Without wrong roundings).
pd.set_option('display.float_format', str)

class bond_pricer:
    
    def __init__(self, 
                 principal_value, 
                 coupon_rate,
                 settlement_date, 
                 maturity_date,
                 coupon_frequency=1,
                 convention='30/360',
                 yield_to_maturity=None, 
                 dirty_price=None,
                 clean_price=None):
        
        self.principal_value = principal_value
        self.coupon_rate = coupon_rate
        self.coupon_frequency = coupon_frequency
        self.coupon_amount = self.principal_value * (self.coupon_rate / self.coupon_frequency)
        self.settlement_date = datetime.strptime(settlement_date, '%d/%m/%Y').date()
        self.maturity_date = datetime.strptime(maturity_date, '%d/%m/%Y').date()
        self.convention = convention
        self.coupon_months = int(12 / self.coupon_frequency)  
        
        # Compute cash flow dates
        cflow_date = self.maturity_date
        cflow_dates = [self.maturity_date]
        while cflow_date + relativedelta(months=-self.coupon_months) > self.settlement_date:
            cflow_date += relativedelta(months=-self.coupon_months)
            cflow_dates.append(cflow_date)        
        self.cash_flow_dates = cflow_dates[::-1]
        
        # Compute accrued interest
        self.next_coupon_date = self.cash_flow_dates[0]
        self.previous_coupon_date = self.next_coupon_date + relativedelta(months=-self.coupon_months)
        if self.convention == 'ACT/ACT':
            self.accrued_days = (self.settlement_date - self.previous_coupon_date).days    
            self.curr_coupon_days = (self.next_coupon_date - self.previous_coupon_date).days
            self.accrual_period = self.accrued_days / self.curr_coupon_days
            self.accrued_interest = self.principal_value * (self.coupon_rate / self.coupon_frequency) * self.accrual_period
        elif self.convention == '30/360':
            d1 = min(30, self.previous_coupon_date.day)
            if d1 == 30:
                d2 = min(d1, self.settlement_date.day)
            else:
                d2 = self.settlement_date.day
            self.accrued_days = 360 * (self.settlement_date.year - self.previous_coupon_date.year) + 30 * (self.settlement_date.month - self.previous_coupon_date.month) + d2 - d1
            self.curr_coupon_days = 360 / self.coupon_frequency
            self.accrual_period = self.accrued_days / self.curr_coupon_days
            self.accrued_interest = self.principal_value * (self.coupon_rate / self.coupon_frequency) * self.accrual_period
        elif self.convention == 'ACT/360':
            self.accrued_days = (self.settlement_date - self.previous_coupon_date).days    
            self.curr_coupon_days = 360 / self.coupon_frequency
            self.accrual_period = self.accrued_days / self.curr_coupon_days
            self.accrued_interest = self.principal_value * (self.coupon_rate / self.coupon_frequency) * self.accrual_period
        elif self.convention == 'ACT/365':
            self.accrued_days = (self.settlement_date - self.previous_coupon_date).days
            self.curr_coupon_days = (self.next_coupon_date - self.previous_coupon_date).days
            self.accrual_period = self.accrued_days / 365
            self.accrued_interest = self.principal_value * (self.coupon_rate / self.coupon_frequency) * self.accrual_period
        
        # Compute cash flow amounts and time to cash flows and maturity
        first_tau = (self.curr_coupon_days - self.accrued_days) / self.curr_coupon_days        
        cash_flow_amounts = []
        taus = []
        for i in range(len(self.cash_flow_dates)):
            cash_flow_amounts.append(self.coupon_amount)
            taus.append((first_tau + i) / self.coupon_frequency)        
        cash_flow_amounts[-1] += self.principal_value
        self.cash_flows = np.array(cash_flow_amounts)
        self.taus = np.array(taus)
        self.time_to_maturity = self.taus[-1]
        
        # Compute clean and dirty prices
        if dirty_price:
            self.dirty_price = dirty_price
            self.clean_price = self.dirty_price - self.accrued_interest
        elif clean_price:
            self.clean_price = clean_price
            self.dirty_price = self.clean_price + self.accrued_interest
        elif yield_to_maturity:
            self.yield_to_maturity = yield_to_maturity
            self.ytm_adjusted = self.coupon_frequency*np.log(1 + self.yield_to_maturity / self.coupon_frequency)
            self.discount_factors = np.exp(-self.ytm_adjusted * self.taus)
            self.dirty_price = np.sum(self.cash_flows * self.discount_factors)
            self.clean_price = self.dirty_price - self.accrued_interest
        
        # Compute current yield
        self.current_yield = (self.coupon_rate * self.principal_value) / self.dirty_price
        
        # Compute yield to maturity
        if yield_to_maturity:
            self.yield_to_maturity = yield_to_maturity
            self.ytm_adjusted = self.coupon_frequency * np.log(1 + self.yield_to_maturity / self.coupon_frequency)
        else:
            def get_bond_price(yield_to_maturity, cash_flows, taus):
                return np.sum(cash_flows * np.exp(-yield_to_maturity * taus))
            
            get_ytm = lambda ytm: get_bond_price(ytm, self.cash_flows, self.taus) - self.dirty_price
            
            self.ytm_adjusted = optimize.newton(get_ytm, 0.04)
            self.yield_to_maturity = (np.exp(self.ytm_adjusted / self.coupon_frequency) - 1) * self.coupon_frequency
            self.discount_factors = np.exp(-self.ytm_adjusted * self.taus)
       
        # Compute duration
        self.first_derivative = np.sum(-self.taus * self.cash_flows * self.discount_factors)
        self.duration_macaulay = -1/self.dirty_price * self.first_derivative
        self.duration_modified = self.duration_macaulay / (1 + self.yield_to_maturity / self.coupon_frequency)                                                        
        
        # Compute convexity
        self.second_derivative = np.sum(self.taus**2 * self.cash_flows * self.discount_factors)
        self.convexity = 1/self.dirty_price * self.second_derivative
        
        # Generate summary table
        index = ['Dirty price','Clean price', 'Accrued interest', 'Face value', 'Coupon rate', 'Coupon frequency', 'Yield to maturity', 
                 'Time to maturity', 'Macaulay duration', 'Modified duration', 'Convexity', 'Settlement date', 'Maturity date', 'Convention']
        data = list(np.round([self.dirty_price, self.clean_price, self.accrued_interest], 7))
        data += list(np.round([self.principal_value, self.coupon_rate, self.coupon_frequency, 
                self.yield_to_maturity, self.time_to_maturity, self.duration_macaulay, self.duration_modified, self.convexity], 4))
        data += [self.settlement_date, self.maturity_date, self.convention]
        self.summary = pd.DataFrame(data, index, columns=[''])
        
    # Approximate new price after delta_ytm change in yield to maturity
    def approx_new_price(self, delta_ytm):
        return self.dirty_price * (1 + (-self.duration_modified * delta_ytm + 0.5 * self.convexity * delta_ytm**2))
    
    # Summary of the bond metrics
    def print_summary(self):
        print(self.summary)

In [2]:
def get_bond_parameters(bond):
    """ Get the necessary parameters for calculations of one single bond 
    
    Parameters:
        bond (pd.Series): Pandas Series object which represents data of the bond.
    
    Returns:
        principal_value (int)
        coupon_rate (float): coupon rate divided by 100 as it is in percents
        coupon_frequency (int)
        settlement_date (str): '%d/%m/%Y'
        maturity_date (str): '%d/%m/%Y'
        yield_to_maturity (float): yield to maturity divided by 100 as it is in percents
        convention (str)
    """
    
    principal_value = bond['Principal Value']
    
    # Coupon rate (%)
    coupon_rate = bond['Coupon Rate (%)'] / 100
    
    coupon_frequency = bond['Coupon Frequency']
    
    # Change date representation from %Y-%m-%d --> %d/%m/%Y
    settlement_date = '/'.join(bond['Settlement Date'].split('-')[::-1]) 
    maturity_date = '/'.join(bond['Maturity Date'].split('-')[::-1])
    
    # Yield to maturity (%)
    yield_to_maturity = bond['Yield to Maturity (%)'] / 100
    
    convention = bond['Convention']
    
    return principal_value, coupon_rate, coupon_frequency, settlement_date, maturity_date, yield_to_maturity, convention


In [3]:
data = pd.read_csv('cleaned_data')

In [4]:
first_bond = data.iloc[0]
first_bond

ISIN                     AMGB1029A235
Coupon Rate (%)                  10.0
Maturity Date              2023-10-29
Dirty Price               104.1177663
Yield to Maturity (%)         11.3308
Coupon Frequency                    2
Principal Value                   100
Settlement Date            2023-04-21
Convention                    ACT/ACT
Name: 0, dtype: object

In [5]:
# Getting the parameters of the first bond in our data
principal_value, coupon_rate, coupon_frequency, settlement_date, maturity_date, yield_to_maturity, convention = get_bond_parameters(first_bond)

In [6]:
print('Parameters of the first bond:')
print('Principal Value:', principal_value)
print('Coupon Rate:', coupon_rate)
print('Coupon Frequency:', coupon_frequency)
print('Settlement Date:', settlement_date)
print('Maturity Date:', maturity_date)
print('Yield to Maturity:', yield_to_maturity)
print('Convention:', convention)

Parameters of the first bond:
Principal Value: 100
Coupon Rate: 0.1
Coupon Frequency: 2
Settlement Date: 21/04/2023
Maturity Date: 29/10/2023
Yield to Maturity: 0.113308
Convention: ACT/ACT


In [16]:
# Instantiate bond
bond = bond_pricer(principal_value=principal_value, coupon_rate=coupon_rate, coupon_frequency=coupon_frequency, 
                settlement_date=settlement_date, maturity_date=maturity_date, 
                yield_to_maturity=yield_to_maturity, convention=convention)

# Print bond metrics
bond.print_summary()

                             
Dirty price        79.3290871
Clean price        78.5390871
Accrued interest         0.79
Face value              100.0
Coupon rate             0.036
Coupon frequency          2.0
Yield to maturity      0.0726
Time to maturity       7.7806
Macaulay duration      6.6413
Modified duration      6.4087
Convexity             48.9532
Settlement date    2023-04-21
Maturity date      2031-02-02
Convention             30/360


In [8]:
data[data['Convention'] == '30/360']

,ISIN,Coupon Rate (%),Maturity Date,Dirty Price,Yield to Maturity (%),Coupon Frequency,Principal Value,Settlement Date,Convention
21,AMEUBDB22ER6,7.15,2025-03-26,39455.4724406,6.364,2,100,2023-04-21,30/360
22,XS2010043904,3.95,2029-09-26,32050.2139503,7.421,2,100,2023-04-21,30/360
23,XS2010028939,3.6,2031-02-02,30717.0158063,7.257,2,100,2023-04-21,30/360


In [9]:
new_bond = data.iloc[23]

In [10]:
new_bond

ISIN                     XS2010028939
Coupon Rate (%)                   3.6
Maturity Date              2031-02-02
Dirty Price             30717.0158063
Yield to Maturity (%)           7.257
Coupon Frequency                    2
Principal Value                   100
Settlement Date            2023-04-21
Convention                     30/360
Name: 23, dtype: object

In [11]:
# Getting the parameters of the '30/360' convention bond in our data
principal_value, coupon_rate, coupon_frequency, settlement_date, maturity_date, yield_to_maturity, convention = get_bond_parameters(new_bond)

In [92]:
print('Parameters of the new bond:')
print('Principal Value:', principal_value)
print('Coupon Rate:', coupon_rate)
print('Coupon Frequency:', coupon_frequency)
print('Settlement Date:', settlement_date)
print('Maturity Date:', maturity_date)
print('Yield to Maturity:', yield_to_maturity)
print('Convention:', convention)

Parameters of the new bond:
Principal Value: 100
Coupon Rate: 0.059500000000000004
Coupon Frequency: 2
Settlement Date: 21/04/2023
Maturity Date: 04/03/2026
Yield to Maturity: 0.075
Convention: ACT/ACT


In [96]:
def show_dirty_prices(bonds):
    """ Shows the dirty prices of our bonds with calculated dirty prices
    
    Parameters:
        bonds (pd.DataFrame): pd.DataFrame object which represents our bonds data.
        
    Returns:
        dirty_prices_comparison (pd.DataFrame): pd.DataFrame object with 2 columns 'Real Dirty Price', 'Calculated Dirty Price'
    """
    
    calculated_dirty_prices = []
    
    for i in range(len(bonds)):
        bond_i = bonds.iloc[i]
        principal_value, coupon_rate, coupon_frequency, settlement_date, maturity_date, yield_to_maturity, convention = get_bond_parameters(bond_i)
        bond = bond_pricer(principal_value=principal_value, coupon_rate=coupon_rate, coupon_frequency=coupon_frequency, 
                    settlement_date=settlement_date, maturity_date=maturity_date, 
                    yield_to_maturity=yield_to_maturity, convention=convention)
        calculated_dirty_prices.append(np.round(bond.dirty_price, 7))
    
    dirty_prices_comparison = pd.DataFrame({'Real Dirty Price': bonds['Dirty Price'], 'Calculated Dirty Price': calculated_dirty_prices})
        
    return dirty_prices_comparison
    

In [97]:
show_dirty_prices(data)

,Real Dirty Price,Calculated Dirty Price
0,104.1177663,104.1177663
1,104.2135116,104.2135116
2,116.6234842,116.6234842
3,113.2018382,113.2018382
4,99.6858725,99.6858725
5,103.6715702,103.7623504
6,110.5905309,110.5905309
7,100.472842,100.472842
8,113.7052612,113.7052612
9,102.9239386,103.0140642
